<a href="https://colab.research.google.com/github/Sid8680/PythonProjects/blob/master/Coursera-week4Assignment_Happy_Sad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()



--2020-07-01 04:52:29--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 172.217.194.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-07-01 04:52:29 (238 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [2]:
happy_dir = os.path.join('/tmp/h-or-s/happy')
sad_dir = os.path.join('/tmp/h-or-s/sad')   

In [3]:
train_happy_dir = os.listdir(happy_dir)
train_sad_dir = os.listdir(sad_dir) 

In [4]:
 model = tf.keras.models.Sequential([
        # Your Code Here
             # Note the input shape is the desired size of the image 150x150 with 3 bytes color
            # This is the first convolution
            tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
            tf.keras.layers.MaxPooling2D(2, 2),
            # The second convolution
            tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            # The third convolution
            tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            # The fourth convolution
            #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
            #tf.keras.layers.MaxPooling2D(2,2),
            # The fifth convolution
            #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
            #tf.keras.layers.MaxPooling2D(2,2),
            # Flatten the results to feed into a DNN
            tf.keras.layers.Flatten(),
            # 256 neuron hidden layer
            tf.keras.layers.Dense(256, activation='relu'),
            # 512 neuron hidden layer
            tf.keras.layers.Dense(512, activation='relu'),
            # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
            tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [6]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen =  ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 80 images belonging to 2 classes.


In [8]:
history = model.fit(
      train_generator,
      steps_per_epoch=1,  
      epochs=20,
      verbose=1,callbacks=[callbacks])      

Epoch 1/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6901 - acc: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 1ms/step - loss: 0.7432 - acc: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 1ms/step - loss: 4.5265 - acc: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 1ms/step - loss: 0.6844 - acc: 0.9625
Epoch 5/20
1/1 [==============================] - 0s 1ms/step - loss: 0.6600 - acc: 0.9375
Epoch 6/20
1/1 [==============================] - 0s 1ms/step - loss: 0.6168 - acc: 0.6625
Epoch 7/20
1/1 [==============================] - 0s 1ms/step - loss: 0.5926 - acc: 0.5500
Epoch 8/20
1/1 [==============================] - 0s 1ms/step - loss: 0.7035 - acc: 0.5000
Epoch 9/20
1/1 [==============================] - 0s 1ms/step - loss: 0.5782 - acc: 0.5000
Epoch 10/20
1/1 [==============================] - 0s 2ms/step - loss: 0.4841 - acc: 0.7500
Epoch 11/20
1/1 [==============================] - 0s 2ms/step - loss: 0.3781 - acc: 0.92

In [9]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a happy smiley")
  else:
    print(fn + " is a sad smiley")

Using TensorFlow backend.


Saving Boy1.jpeg to Boy1 (2).jpeg
[1.]
Boy1.jpeg is a happy smiley


In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)